# Detecting Midjourney Images via Feature Engineering & Classification

**Objectives**

- **Engineer Discriminative Features**
    Develop and extract features that effectively separate Stable Diffusion (AI-generated) images from authentic camera-captured photos using frequency, color, and texture analysis

- **Build Reproducible Pipeline**

    Construct a robust feature extraction pipeline that generates a tabular dataset suitable for ML. Ensure consistency and reproducibility across experiments.

- **Train & Evaluate Classifiers**

    Implement a classifier architectures such as Random Forest, XGBoost, SVM or Neural Networks. Rigorously evaluate performance using industry-standard metrics and confusion matrices

- **Quantify Feature Relevance**

    Apply multiple interpretability techniques such as Gini importance. Permutation importance, and SHAP values to understand which features drive classification decisions.


**Dataset Structure**
```bash
imagenet_midjourney/
|----test/ 
| |----ai/ 
| | |--[AI-generated images] (Stable Diffusion/Midjourney) 
| | Label: 1 (fake) 
| |----nature/ 
| | |--[Natural camera images] 
| | (Non-AI photographs) 
| | Label: 0 (real)
```

## Imports

In [42]:
import os
import cv2
import numpy as np
import pandas as pd

from skimage import io, color, img_as_ubyte
from tqdm import tqdm

from scipy.stats import linregress, gmean, pearsonr, kurtosis

from skimage.feature import graycomatrix, graycoprops

DATASET_DIR = "imagenet_midjourney/test"

CATEGORIES = {
    "ai": 1,
    "nature": 0
}

IMG_SIZE = (256, 256)
COLOR_MODE = 'rgb'

## Image Loading & Preprocessing

Load images from dataset directories, normalize formats, and prepare for feature extraction. Handle various input formats consistently.

In [11]:
def load_and_preprocess_images(base_dir, categories, img_size=(256, 256), color_mode='rgb'):
    X, y, paths = [], [], []

    for category, label in categories.items():
        folder = os.path.join(base_dir, category)
        if not os.path.exists(folder):
            print(f"Folder not encountered: {folder}")
            continue

        for filename in tqdm(os.listdir(folder), desc=f"Loading {category}"):
            file_path = os.path.join(folder, filename)

            if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
                continue

            try:
                image = io.imread(file_path)

                if color_mode == 'rgb':
                    if image.ndim == 2:
                        image = color.gray2rgb(image)
                    elif image.shape[2] == 4:
                        image = color.rgba2rgb(image)
                elif color_mode == 'gray':
                    image = color.rgb2gray(image)

                image = cv2.resize(img_as_ubyte(image), img_size)

                image = image.astype(np.float32) / 255.0

                X.append(image)
                y.append(label)
                paths.append(file_path)
                
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
                continue

    X = np.array(X)
    y = np.array(y)
    return X, y, paths

In [12]:
X, y, image_paths = load_and_preprocess_images(DATASET_DIR, CATEGORIES, IMG_SIZE, COLOR_MODE)

print(f"Total imágenes cargadas: {len(X)}")
print(f"Dimensión de ejemplo: {X[0].shape}")

Loading nature: 100%|██████████| 500/500 [00:02<00:00, 196.48it/s]


Total imágenes cargadas: 1000
Dimensión de ejemplo: (256, 256, 3)


## Feature Computation

Extract per-image features across frequency, color, and texture domains. Generate comprehensive feature vectors for each sample.

### Feature Family I: Frequency & Spectrum Analysis (FFT)

In [14]:
def compute_fft_features(image):
    gray = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    gray = gray.astype(np.float32) / 255.0

    fft2 = np.fft.fft2(gray)
    fshift = np.fft.fftshift(fft2)
    magnitude_spectrum = np.abs(fshift) ** 2

    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2
    y, x = np.ogrid[:rows, :cols]
    radius = np.sqrt((x - ccol) ** 2 + (y - crow) ** 2).astype(np.int32)

    radial_profile = np.bincount(radius.ravel(), magnitude_spectrum.ravel()) / np.bincount(radius.ravel())
    radial_profile = radial_profile[1:]

    radial_power_spectrum_mean = np.mean(radial_profile)

    freqs = np.arange(1, len(radial_profile) + 1)
    log_freqs = np.log(freqs)
    log_power = np.log(radial_profile + 1e-8)
    slope, intercept, _, _, _ = linregress(log_freqs, log_power)
    spectral_slope = slope

    spectral_flatness = gmean(radial_profile + 1e-8) / (np.mean(radial_profile) + 1e-8)

    cutoff = len(radial_profile) // 3
    high_freq_energy = np.sum(radial_profile[-cutoff:])
    total_energy = np.sum(radial_profile)
    high_freq_ratio = high_freq_energy / (total_energy + 1e-8)

    return {
        'radial_power_spectrum_mean': radial_power_spectrum_mean,
        'spectral_slope': spectral_slope,
        'spectral_flatness': spectral_flatness,
        'high_freq_ratio': high_freq_ratio
    }


### Feature Family II: Colo & Chrominance Analysis

In [29]:
def safe_pearsonr(a, b):
    if np.std(a) < 1e-8 or np.std(b) < 1e-8:
        return 0.0
    r, _ = pearsonr(a, b)
    return np.nan_to_num(r, nan=0.0, posinf=0.0, neginf=0.0)

def safe_kurtosis(x):
    if x.size == 0 or np.allclose(x, x.flat[0], atol=1e-8) or np.std(x) < 1e-8:
        return 0.0
    val = kurtosis(x, fisher=False)
    return np.nan_to_num(val, nan=0.0, posinf=0.0, neginf=0.0)

def compute_color_features(image):
    img_8bit = (image * 255).astype(np.uint8)

    R = img_8bit[:, :, 0].astype(np.float32).ravel()
    G = img_8bit[:, :, 1].astype(np.float32).ravel()
    B = img_8bit[:, :, 2].astype(np.float32).ravel()

    corr_rg = safe_pearsonr(R, G)
    corr_rb = safe_pearsonr(R, B)
    corr_gb = safe_pearsonr(G, B)
    
    rgb_corr_mean = np.mean([corr_rg, corr_rb, corr_gb])
    rgb_corr_std = np.std([corr_rg, corr_rb, corr_gb])

    ycbcr = cv2.cvtColor(img_8bit, cv2.COLOR_RGB2YCrCb)
    Y, Cr, Cb = cv2.split(ycbcr)  
    
    cb_kurt = safe_kurtosis(Cb.ravel())
    cr_kurt = safe_kurtosis(Cr.ravel())

    cb_lap = cv2.Laplacian(Cb.astype(np.float32), cv2.CV_32F, ksize=3)
    cr_lap = cv2.Laplacian(Cr.astype(np.float32), cv2.CV_32F, ksize=3)

    cb_lap = np.clip(cb_lap, -5000, 5000)
    cr_lap = np.clip(cr_lap, -5000, 5000)

    cb_residual_kurt = safe_kurtosis(cb_lap.ravel())
    cr_residual_kurt = safe_kurtosis(cr_lap.ravel())

    return {
        'rgb_corr_mean': rgb_corr_mean,
        'rgb_corr_std': rgb_corr_std,
        'cb_kurtosis': cb_kurt,
        'cr_kurtosis': cr_kurt,
        'cb_residual_kurtosis': cb_residual_kurt,
        'cr_residual_kurtosis': cr_residual_kurt
    }

### Feature Family III: 

In [44]:
def compute_glcm_features(image, levels=64, distances=[1, 2, 4], angles=None):

    if angles is None:
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

    gray = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)

    gray_q = np.floor(gray / (256 / levels)).astype(np.uint8)

    glcm = graycomatrix(
        gray_q,
        distances=distances,
        angles=angles,
        levels=levels,
        symmetric=True,
        normed=True
    )

    props = {}
    for prop in ['contrast', 'homogeneity', 'energy', 'correlation']:
        vals = graycoprops(glcm, prop)
        props[prop] = np.mean(vals)

    for k in props:
        if np.isnan(props[k]) or np.isinf(props[k]):
            props[k] = 0.0

    return props

## Tabular Dataset Creation

In [45]:
fft_feature_list = []
color_features_list = []
glcm_feature_list = []

for img in tqdm(X, desc="Extracting features"):
    fft = compute_fft_features(img)
    color = compute_color_features(img)
    texture = compute_glcm_features(img)

    fft_feature_list.append(list(fft.values()))
    color_features_list.append(list(color.values()))
    glcm_feature_list.append(list(texture.values()))

fft_features = np.array(fft_feature_list)
color_features = np.array(color_features_list)
texture_features = np.array(glcm_feature_list)

fft_features = np.nan_to_num(fft_features, nan=0.0, posinf=0.0, neginf=0.0)
color_features = np.nan_to_num(color_features, nan=0.0, posinf=0.0, neginf=0.0)
texture_features = np.nan_to_num(texture_features, nan=0.0, posinf=0.0, neginf=0.0)

print(f"FFT features shape: {fft_features.shape}")
print(f"Color features shape: {color_features.shape}")
print(f"GLCM features shape: {texture_features.shape}")
print("NaN restantes:", np.isnan(texture_features).sum())

Extracting features: 100%|██████████| 1000/1000 [00:17<00:00, 57.00it/s]

FFT features shape: (1000, 4)
Color features shape: (1000, 6)
GLCM features shape: (1000, 4)
NaN restantes: 0


In [46]:
X_features = np.concatenate([fft_features, color_features, texture_features], axis=1)

print(f"Feature matrix shape: {X_features.shape}")

Feature matrix shape: (1000, 14)


In [47]:
fft_cols = list(fft.keys())
color_cols = list(color.keys())
texture_cols = list(texture.keys())

columns = fft_cols + color_cols + texture_cols

df_features = pd.DataFrame(X_features, columns=columns)
df_features['label'] = y

df_features.to_csv("image_features_dataset.csv", index=False)
print("✅ Dataset tabular creado y guardado en 'image_features_dataset.csv'")
print(df_features.head())

✅ Dataset tabular creado y guardado en 'image_features_dataset.csv'
   radial_power_spectrum_mean  spectral_slope  spectral_flatness  \
0                63553.043900       -2.032502           0.011904   
1                72022.384206       -2.275228           0.016276   
2                44560.433672       -2.825358           0.005280   
3                96580.610755       -2.394801           0.007560   
4                46259.113538       -2.570078           0.008452   

   high_freq_ratio  rgb_corr_mean  rgb_corr_std  cb_kurtosis  cr_kurtosis  \
0         0.001163       0.980505      0.011808     5.466961     3.621634   
1         0.001194       0.961580      0.022610     3.250981     3.333180   
2         0.000199       0.934480      0.040921     7.161317     5.584515   
3         0.000367       0.731766      0.180140     3.528350     3.422427   
4         0.000408       0.804772      0.011807     3.416708     2.300165   

   cb_residual_kurtosis  cr_residual_kurtosis    contrast  h

## Train/Test Splitting